# Лабораторная 2 - 10 баллов (дедлайн 14.03)
## Задача классификации цветов автомобилей.
### Датасет DVM: https://deepvisualmarketing.github.io/ фронтальный!
## Ход работы:
Написать своими руками классификатор любой на выбор: ResNet, InceptionV3, DenseNet, MobileNet, ShuffleNet и обучить его на полученном датасете.

Также взять аналогичный классификатор, но предобученный на ImageNet или на Cityscapes и дообучить на собранном датасете. Выяснить, чей классификатор лучше.

Оценка качества производится при помощи F1_macro, требуется получить F1_macro > 0.8.

### Сравнить полученное качество и сделать вывод.

## Шаг №0 - Импорт библиотек

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import compute_class_weight
from tensorflow import keras
import tensorflow as tf
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import seaborn as sns
from collections import Counter

## Шаг №1 - Функции для загрузки и предварительной обработки данных

In [ ]:
def ensure_class_representation(train_df, val_df):
    """
    Функция для проверки и обеспечения равномерного распределения классов в обучающей и валидационной выборках
    Description: Исключает данный из обучающей выборки, если такой класс отсутствует в валидационной выборке
    """
    val_classes = Counter(val_df['color'])
    train_classes = Counter(train_df['color'])

    for cls in train_classes:
        if cls not in val_classes or val_classes[cls] == 0:
            samples = train_df[train_df['color'] == cls].sample(1)
            if not samples.empty:
                val_df = pd.concat([val_df, samples])
                train_df = train_df.drop(samples.index)

    return train_df, val_df


def prepare_data(data_dir, batch_size=32, img_size=(299, 299), min_samples=500, excluded_colors=None):
    """
    Функция для подготовки данных для обучения модели
    Description: Загружает изображения из директории, разделяет их на обучающую, валидационную и тестовую выборки,
    обеспечивает равномерное распределение классов, создает генераторы для обучения, валидации и тестирования
    """
    image_paths = []
    colors = []

    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.jpg'):
                full_path = os.path.join(root, file)
                try:
                    color = file.split('$$')[3]
                    image_paths.append(full_path)
                    colors.append(color)
                except IndexError:
                    print(f"Пропускаем файл с неправильным форматом: {file}")

    df = pd.DataFrame({
        'image_path': image_paths,
        'color': colors
    })

    print(f"Распределение цветов в исходном датасете:")
    print(df['color'].value_counts())
    print(f"Количество классов: {df['color'].nunique()}")

    if excluded_colors is None:
        excluded_colors = ['Multicolour', 'Unlisted']

    df = df[~df['color'].isin(excluded_colors)]

    color_counts = df['color'].value_counts()
    valid_colors = color_counts[color_counts >= min_samples].index.tolist()

    df = df[df['color'].isin(valid_colors)]

    normalized_dfs = []
    for color in valid_colors:
        color_df = df[df['color'] == color]
        if len(color_df) > min_samples:
            normalized_dfs.append(color_df.sample(min_samples, random_state=42))
        else:
            normalized_dfs.append(color_df)

    df = pd.concat(normalized_dfs)

    print(f"Распределение цветов в отфильтрованном и нормализованном датасете:")
    print(df['color'].value_counts())
    print(f"Количество классов: {df['color'].nunique()}")

    from sklearn.model_selection import train_test_split

    try:
        train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['color'], random_state=42)
        train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['color'],
                                            random_state=42)
    except ValueError as e:
        print(f"Предупреждение: {e}")
        print("Переключаемся на нестратифицированное разделение данных")
        train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
        train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)

    train_df, val_df = ensure_class_representation(train_df, val_df)

    print(f"Размер обучающей выборки: {len(train_df)}")
    print(f"Размер валидационной выборки: {len(val_df)}")
    print(f"Размер тестовой выборки: {len(test_df)}")

    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        brightness_range=[0.9, 1.1],
        fill_mode='nearest',
    )

    val_test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='image_path',
        y_col='color',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    validation_generator = val_test_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='image_path',
        y_col='color',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    test_generator = val_test_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='image_path',
        y_col='color',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, validation_generator, test_generator, train_generator.class_indices

## Шаг №2 - Определение класса для создания модели

In [ ]:
class CarColorClassifier:
    """
    Класс для создания модели классификатора цветов автомобилей
    """
    def __init__(self, input_shape=(299, 299, 3), num_classes=None, useGPU=False):
        """
        Функция инициализации
        :param input_shape: размер входного изображения
        :param num_classes: количество классов для классификации
        :param useGPU: флаг использования GPU
        """
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = None
        self.useGPU = useGPU

        if self.useGPU:
            physical_devices = tf.config.list_physical_devices('GPU')
            if len(physical_devices) > 0:
                print(f"Найдено {len(physical_devices)} GPU устройств. Будет использоваться GPU.")
                for device in physical_devices:
                    tf.config.experimental.set_memory_growth(device, True)
            else:
                print("GPU не обнаружен, будет использоваться CPU несмотря на параметр useGPU=True.")
                self.useGPU = False
        else:
            print("Выбран режим CPU.")
            os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

    def build_inception_from_scratch(self):
        """
        Функция для создания модели InceptionV3 с нуля
        """
        with tf.device('/GPU:0' if self.useGPU else '/CPU:0'):
            inputs = keras.layers.Input(shape=self.input_shape)

            x = keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding='valid')(inputs)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)
            x = keras.layers.Conv2D(64, (3, 3), padding='valid')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)
            x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

            for i in range(5):
                filter_scale = min(1 + i * 0.3, 2.5)

                branch1x1 = keras.layers.Conv2D(int(64 * filter_scale), (1, 1), padding='same', activation='relu')(x)

                branch5x5 = keras.layers.Conv2D(int(48 * filter_scale), (1, 1), padding='same', activation='relu')(x)
                branch5x5 = keras.layers.Conv2D(int(64 * filter_scale), (5, 5), padding='same', activation='relu')(
                    branch5x5)

                branch3x3dbl = keras.layers.Conv2D(int(64 * filter_scale), (1, 1), padding='same', activation='relu')(x)
                branch3x3dbl = keras.layers.Conv2D(int(96 * filter_scale), (3, 3), padding='same', activation='relu')(
                    branch3x3dbl)
                branch3x3dbl = keras.layers.Conv2D(int(96 * filter_scale), (3, 3), padding='same', activation='relu')(
                    branch3x3dbl)

                branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
                branch_pool = keras.layers.Conv2D(int(64 * filter_scale), (1, 1), padding='same', activation='relu')(
                    branch_pool)

                x = keras.layers.Concatenate(axis=-1)([branch1x1, branch5x5, branch3x3dbl, branch_pool])

                x = keras.layers.BatchNormalization()(x)

                if i >= 2:
                    x = keras.layers.Dropout(0.1)(x)

                if (i + 1) % 2 == 0 and i < 4:
                    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

            x = keras.layers.GlobalAveragePooling2D()(x)

            x = keras.layers.Dense(1024, activation='relu')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Dropout(0.5)(x)

            x = keras.layers.Dense(512, activation='relu')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Dropout(0.3)(x)

            x = keras.layers.Dense(256, activation='relu')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Dropout(0.2)(x)

            x = keras.layers.Dense(self.num_classes, activation='softmax')(x)

            model = keras.Model(inputs, x)
            return model

    def build_pretrained_model(self):
        """
        Функция для создания предобученной модели InceptionV3
        """
        with tf.device('/GPU:0' if self.useGPU else '/CPU:0'):
            base_model = keras.applications.InceptionV3(
                include_top=False,
                weights='imagenet',
                input_shape=self.input_shape
            )

            x = base_model.output
            x = keras.layers.GlobalAveragePooling2D()(x)
            x = keras.layers.Dense(1024, activation='relu')(x)
            x = keras.layers.Dropout(0.5)(x)
            x = keras.layers.Dense(512, activation='relu')(x)
            x = keras.layers.Dropout(0.3)(x)
            x = keras.layers.Dense(self.num_classes, activation='softmax')(x)

            model = keras.Model(inputs=base_model.input, outputs=x)

            for layer in base_model.layers:
                layer.trainable = False

            return model

    def compile_model(self, learning_rate=0.0005):
        """
        Функция для компиляции модели
        """
        with tf.device('/GPU:0' if self.useGPU else '/CPU:0'):
            optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
            self.model.compile(
                optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )

    def train(self, train_generator, validation_generator, epochs=50, callbacks=None, class_weight=None):
        """
        Функция для обучения модели
        """
        with tf.device('/GPU:0' if self.useGPU else '/CPU:0'):
            history = self.model.fit(
                train_generator,
                validation_data=validation_generator,
                epochs=epochs,
                callbacks=callbacks,
                class_weight=class_weight,
                verbose=1
            )
            return history

    def evaluate(self, test_generator):
        """
        Функция для оценки качества модели
        """
        with tf.device('/GPU:0' if self.useGPU else '/CPU:0'):
            return self.model.evaluate(test_generator)

    def predict(self, test_generator):
        """
        Функция для предсказания классов
        """
        with tf.device('/GPU:0' if self.useGPU else '/CPU:0'):
            return self.model.predict(test_generator)

## Шаг №3 - Определение класса для вычисления F1 макро

In [ ]:
class F1Callback(keras.callbacks.Callback):
    """
    Класс для вычисления F1 макро на валидационной выборке
    """
    def __init__(self, validation_data):
        """
        Функция инициализации
        :param validation_data: данные для валидации
        """
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        """
        Функция для вычисления F1 макро на валидационной выборке
        """
        x_val, y_val = self.validation_data
        y_pred = np.argmax(self.model.predict(x_val), axis=1)
        y_true = np.argmax(y_val, axis=1)
        f1 = f1_score(y_true, y_pred, average='macro')
        print(f' - val_f1_macro: {f1:.4f}')
        logs['val_f1_macro'] = f1

## Шаг №4 - Определение функций для визуализации результатов

In [ ]:
def evaluate_model(model, test_generator, class_indices):
    """
    Функция для оценки качества модели и построения матрицы ошибок
    """
    predictions = model.predict(test_generator)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_generator.classes

    f1_macro = f1_score(y_true, y_pred, average='macro')

    class_labels = {v: k for k, v in class_indices.items()}
    target_names = [class_labels[i] for i in range(len(class_labels))]

    print("\nКлассификационный отчет:")
    print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

    print(f"\nF1 Macro: {f1_macro:.4f}")

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
    plt.title('Матрица ошибок')
    plt.ylabel('Истинный класс')
    plt.xlabel('Предсказанный класс')
    plt.tight_layout()
    plt.show()

    return f1_macro


def plot_training_history(history):
    """
    Функция для визуализации истории обучения модели и построения графиков
    """
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b-', label='Training Loss')
    plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

## Шаг №5 - Определение вспомогательной функции для валидации скорости обучения в зависимости от количества эпох

In [ ]:
def lr_schedule(epoch, initial_lr=0.0005):
    """
    Функция для планирования скорости обучения
    """
    lr = initial_lr
    if epoch > 30:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.5
    return lr

## Шаг №6 - Определение поведения (эксперимента) кастомной модели

In [ ]:
def run_experiment_custom(data_dir, batch_size=32, img_size=(299, 299), epochs=30, learning_rate=0.0005, use_gpu=False):
    """
    Функция для запуска эксперимента с кастомной моделью
    """
    print("\n=== Эксперимент с моделью InceptionV3 с нуля ===")
    print(f"Режим обучения: {'GPU' if use_gpu else 'CPU'}")

    train_generator, validation_generator, test_generator, class_indices = prepare_data(
        data_dir, batch_size, img_size
    )

    num_classes = len(class_indices)
    classifier = CarColorClassifier(input_shape=(img_size[0], img_size[1], 3), num_classes=num_classes, useGPU=use_gpu)
    classifier.model = classifier.build_inception_from_scratch()
    classifier.compile_model(learning_rate=learning_rate)

    val_data = next(validation_generator)
    f1_callback = F1Callback(val_data)

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True, verbose=1),
        ModelCheckpoint('best_custom_model.keras', save_best_only=True, monitor='val_loss', verbose=1),
        keras.callbacks.LearningRateScheduler(lr_schedule),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        ),
        f1_callback
    ]

    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_generator.classes),
        y=train_generator.classes
    )
    class_weights_dict = dict(enumerate(class_weights))

    history = classifier.train(
        train_generator,
        validation_generator,
        epochs=epochs,
        callbacks=callbacks,
        class_weight=class_weights_dict
    )

    plot_training_history(history)

    f1_macro = evaluate_model(classifier.model, test_generator, class_indices)

    return classifier.model, f1_macro

## Шаг №7 - Определение поведения (эксперимента) предобученной модели

In [ ]:
def run_experiment_pretrained(data_dir, batch_size=32, img_size=(299, 299), epochs=30, learning_rate=0.0005, use_gpu=False):
    """
    Функция для запуска эксперимента с предобученной моделью
    """
    print("\n=== Эксперимент с предобученной моделью InceptionV3 ===")
    print(f"Режим обучения: {'GPU' if use_gpu else 'CPU'}")

    train_generator, validation_generator, test_generator, class_indices = prepare_data(
        data_dir, batch_size, img_size
    )

    num_classes = len(class_indices)
    classifier = CarColorClassifier(input_shape=(img_size[0], img_size[1], 3), num_classes=num_classes, useGPU=use_gpu)
    classifier.model = classifier.build_pretrained_model()
    classifier.compile_model(learning_rate=learning_rate)

    val_data = next(validation_generator)
    f1_callback = F1Callback(val_data)

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1),
        ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', verbose=1),
        keras.callbacks.LearningRateScheduler(lr_schedule),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        )
    ]

    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_generator.classes),
        y=train_generator.classes
    )
    class_weights_dict = dict(enumerate(class_weights))

    history = classifier.train(
        train_generator,
        validation_generator,
        epochs=epochs,
        callbacks=callbacks,
        class_weight=class_weights_dict
    )

    plot_training_history(history)

    print("\nДообучение с разблокировкой последних слоев...")
    for layer in classifier.model.layers[-20:]:
        layer.trainable = True

    classifier.compile_model(learning_rate=learning_rate / 10)

    history_fine = classifier.train(
        train_generator,
        validation_generator,
        epochs=100,
        callbacks=callbacks,
        class_weight=class_weights_dict
    )

    plot_training_history(history_fine)

    f1_macro = evaluate_model(classifier.model, test_generator, class_indices)

    return classifier.model, f1_macro

## Шаг №8 - Определение начальных параметров

In [ ]:
data_dir = "dataset"

batch_size = 32
img_size = (299, 299)
epochs = 100
learning_rate = 0.00005
use_gpu = True

## Шаг №9 - Запуск экспериментов

In [ ]:
print(f"\n=== Начало эксперимента ===")
print(f"Использование GPU: {use_gpu}")

if use_gpu:
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print(f"Доступно GPU устройств: {len(physical_devices)}")
        print(f"Устройства: {physical_devices}")
    else:
        print("GPU не обнаружен. Рекомендуется установить параметр use_gpu=False.")

custom_model, custom_f1 = run_experiment_custom(
    data_dir, batch_size, img_size, epochs, learning_rate, use_gpu
)

pretrained_model, pretrained_f1 = run_experiment_pretrained(
    data_dir, batch_size, img_size, epochs, learning_rate, use_gpu
)

## Шаг №10 - Сравнение результатов

In [ ]:
print("\n=== Сравнение результатов ===")
print(f"F1 Macro модели с нуля: {custom_f1:.4f}")
print(f"F1 Macro предобученной модели: {pretrained_f1:.4f}")

if pretrained_f1 >= custom_f1:
    print("Предобученная модель показала лучшие результаты.")
else:
    print("Модель, обученная с нуля, показала лучшие результаты.")

## Вывод для первого эксперимента
Вывод скопировал из консоли домашнего компьютера, тк там не запустился jupiter notebook, а на ноутбуке было грустно из-за отсутствия видеокарты


In [ ]:
"""
=== Эксперимент с моделью InceptionV3 с нуля ===
Режим обучения: GPU
Распределение цветов в исходном датасете:
color
Black          14317
Grey            9474
White           9395
Blue            8483
Silver          7770
Red             6095
Unlisted        1516
Brown            911
Green            777
Yellow           667
Beige            600
Orange           559
Purple           362
Bronze           329
Gold             217
Multicolour      196
Pink              87
Turquoise         26
Maroon            26
Burgundy           9
Magenta            9
Navy               1
Indigo             1
Name: count, dtype: int64
Количество классов: 23
Распределение цветов в отфильтрованном и нормализованном датасете:
color
Black     500
Grey      500
White     500
Blue      500
Silver    500
Red       500
Brown     500
Green     500
Yellow    500
Beige     500
Orange    500
Name: count, dtype: int64
Количество классов: 11
Размер обучающей выборки: 3520
Размер валидационной выборки: 880
Размер тестовой выборки: 1100
Found 3520 validated image filenames belonging to 11 classes.
Found 880 validated image filenames belonging to 11 classes.
Found 1100 validated image filenames belonging to 11 classes.
Найдено 1 GPU устройств. Будет использоваться GPU.
Epoch 1/100
  6/110 [>.............................] - ETA: 30s - loss: 3.0701 - accuracy: 0.1042WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1135s vs `on_train_batch_end` time: 0.1468s). Check your callbacks.
110/110 [==============================] - ETA: 0s - loss: 2.3827 - accuracy: 0.2665
Epoch 1: val_loss improved from inf to 2.50598, saving model to best_custom_model.keras
1/1 [==============================] - 0s 380ms/step
 - val_f1_macro: 0.0076
110/110 [==============================] - 60s 461ms/step - loss: 2.3827 - accuracy: 0.2665 - val_loss: 2.5060 - val_accuracy: 0.0352 - lr: 5.0000e-05 - val_f1_macro: 0.0076
Epoch 2/100
110/110 [==============================] - ETA: 0s - loss: 1.9502 - accuracy: 0.3724
Epoch 2: val_loss did not improve from 2.50598
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.0214
110/110 [==============================] - 54s 488ms/step - loss: 1.9502 - accuracy: 0.3724 - val_loss: 2.5824 - val_accuracy: 0.0898 - lr: 5.0000e-05 - val_f1_macro: 0.0214
Epoch 3/100
110/110 [==============================] - ETA: 0s - loss: 1.7580 - accuracy: 0.4168
Epoch 3: val_loss did not improve from 2.50598
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.0375
110/110 [==============================] - 54s 492ms/step - loss: 1.7580 - accuracy: 0.4168 - val_loss: 2.6778 - val_accuracy: 0.0977 - lr: 5.0000e-05 - val_f1_macro: 0.0375
Epoch 4/100
110/110 [==============================] - ETA: 0s - loss: 1.6219 - accuracy: 0.4639
Epoch 4: val_loss did not improve from 2.50598
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.1564
110/110 [==============================] - 54s 490ms/step - loss: 1.6219 - accuracy: 0.4639 - val_loss: 2.5068 - val_accuracy: 0.1807 - lr: 5.0000e-05 - val_f1_macro: 0.1564
Epoch 5/100
110/110 [==============================] - ETA: 0s - loss: 1.5780 - accuracy: 0.4807
Epoch 5: val_loss improved from 2.50598 to 1.55190, saving model to best_custom_model.keras
1/1 [==============================] - 0s 28ms/step
 - val_f1_macro: 0.2901
110/110 [==============================] - 55s 496ms/step - loss: 1.5780 - accuracy: 0.4807 - val_loss: 1.5519 - val_accuracy: 0.4045 - lr: 5.0000e-05 - val_f1_macro: 0.2901
Epoch 6/100
110/110 [==============================] - ETA: 0s - loss: 1.4558 - accuracy: 0.5060
Epoch 6: val_loss improved from 1.55190 to 1.38036, saving model to best_custom_model.keras
1/1 [==============================] - 0s 28ms/step
 - val_f1_macro: 0.3230
110/110 [==============================] - 56s 504ms/step - loss: 1.4558 - accuracy: 0.5060 - val_loss: 1.3804 - val_accuracy: 0.5000 - lr: 5.0000e-05 - val_f1_macro: 0.3230
Epoch 7/100
110/110 [==============================] - ETA: 0s - loss: 1.4213 - accuracy: 0.5122
Epoch 7: val_loss improved from 1.38036 to 1.32663, saving model to best_custom_model.keras
1/1 [==============================] - 0s 26ms/step
 - val_f1_macro: 0.3964
110/110 [==============================] - 55s 498ms/step - loss: 1.4213 - accuracy: 0.5122 - val_loss: 1.3266 - val_accuracy: 0.5295 - lr: 5.0000e-05 - val_f1_macro: 0.3964
Epoch 8/100
110/110 [==============================] - ETA: 0s - loss: 1.3682 - accuracy: 0.5352
Epoch 8: val_loss did not improve from 1.32663
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.3384
110/110 [==============================] - 54s 489ms/step - loss: 1.3682 - accuracy: 0.5352 - val_loss: 1.5625 - val_accuracy: 0.4716 - lr: 5.0000e-05 - val_f1_macro: 0.3384
Epoch 9/100
110/110 [==============================] - ETA: 0s - loss: 1.3207 - accuracy: 0.5500
Epoch 9: val_loss improved from 1.32663 to 1.26143, saving model to best_custom_model.keras
1/1 [==============================] - 0s 28ms/step
 - val_f1_macro: 0.3325
110/110 [==============================] - 55s 500ms/step - loss: 1.3207 - accuracy: 0.5500 - val_loss: 1.2614 - val_accuracy: 0.5580 - lr: 5.0000e-05 - val_f1_macro: 0.3325
Epoch 10/100
110/110 [==============================] - ETA: 0s - loss: 1.3340 - accuracy: 0.5543
Epoch 10: val_loss did not improve from 1.26143
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4333
110/110 [==============================] - 54s 489ms/step - loss: 1.3340 - accuracy: 0.5543 - val_loss: 1.3434 - val_accuracy: 0.5307 - lr: 5.0000e-05 - val_f1_macro: 0.4333
Epoch 11/100
110/110 [==============================] - ETA: 0s - loss: 1.2746 - accuracy: 0.5685
Epoch 11: val_loss improved from 1.26143 to 1.15693, saving model to best_custom_model.keras
1/1 [==============================] - 0s 28ms/step
 - val_f1_macro: 0.5207
110/110 [==============================] - 56s 505ms/step - loss: 1.2746 - accuracy: 0.5685 - val_loss: 1.1569 - val_accuracy: 0.6193 - lr: 5.0000e-05 - val_f1_macro: 0.5207
Epoch 12/100
110/110 [==============================] - ETA: 0s - loss: 1.2171 - accuracy: 0.5827
Epoch 12: val_loss did not improve from 1.15693
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4116
110/110 [==============================] - 54s 489ms/step - loss: 1.2171 - accuracy: 0.5827 - val_loss: 1.2626 - val_accuracy: 0.5659 - lr: 5.0000e-05 - val_f1_macro: 0.4116
Epoch 13/100
110/110 [==============================] - ETA: 0s - loss: 1.1745 - accuracy: 0.5835
Epoch 13: val_loss did not improve from 1.15693
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.5428
110/110 [==============================] - 54s 489ms/step - loss: 1.1745 - accuracy: 0.5835 - val_loss: 1.2440 - val_accuracy: 0.5966 - lr: 5.0000e-05 - val_f1_macro: 0.5428
Epoch 14/100
110/110 [==============================] - ETA: 0s - loss: 1.1374 - accuracy: 0.6057
Epoch 14: val_loss did not improve from 1.15693
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.3994
110/110 [==============================] - 54s 488ms/step - loss: 1.1374 - accuracy: 0.6057 - val_loss: 1.1735 - val_accuracy: 0.5739 - lr: 5.0000e-05 - val_f1_macro: 0.3994
Epoch 15/100
110/110 [==============================] - ETA: 0s - loss: 1.1154 - accuracy: 0.6139
Epoch 15: val_loss did not improve from 1.15693
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4167
110/110 [==============================] - 54s 490ms/step - loss: 1.1154 - accuracy: 0.6139 - val_loss: 1.5063 - val_accuracy: 0.4670 - lr: 5.0000e-05 - val_f1_macro: 0.4167
Epoch 16/100
110/110 [==============================] - ETA: 0s - loss: 1.0820 - accuracy: 0.6239
Epoch 16: val_loss improved from 1.15693 to 1.15474, saving model to best_custom_model.keras
1/1 [==============================] - 0s 28ms/step
 - val_f1_macro: 0.5100
110/110 [==============================] - 55s 502ms/step - loss: 1.0820 - accuracy: 0.6239 - val_loss: 1.1547 - val_accuracy: 0.5886 - lr: 5.0000e-05 - val_f1_macro: 0.5100
Epoch 17/100
110/110 [==============================] - ETA: 0s - loss: 1.0740 - accuracy: 0.6244
Epoch 17: val_loss did not improve from 1.15474
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4224
110/110 [==============================] - 54s 491ms/step - loss: 1.0740 - accuracy: 0.6244 - val_loss: 1.1935 - val_accuracy: 0.5750 - lr: 5.0000e-05 - val_f1_macro: 0.4224
Epoch 18/100
110/110 [==============================] - ETA: 0s - loss: 1.0634 - accuracy: 0.6315
Epoch 18: val_loss did not improve from 1.15474
1/1 [==============================] - 0s 29ms/step
 - val_f1_macro: 0.3743
110/110 [==============================] - 55s 494ms/step - loss: 1.0634 - accuracy: 0.6315 - val_loss: 1.3696 - val_accuracy: 0.5534 - lr: 5.0000e-05 - val_f1_macro: 0.3743
Epoch 19/100
110/110 [==============================] - ETA: 0s - loss: 1.0010 - accuracy: 0.6474
Epoch 19: val_loss did not improve from 1.15474
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4128
110/110 [==============================] - 55s 495ms/step - loss: 1.0010 - accuracy: 0.6474 - val_loss: 1.2375 - val_accuracy: 0.5841 - lr: 5.0000e-05 - val_f1_macro: 0.4128
Epoch 20/100
110/110 [==============================] - ETA: 0s - loss: 0.9954 - accuracy: 0.6554
Epoch 20: val_loss did not improve from 1.15474
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4529
110/110 [==============================] - 55s 495ms/step - loss: 0.9954 - accuracy: 0.6554 - val_loss: 1.3582 - val_accuracy: 0.5886 - lr: 5.0000e-05 - val_f1_macro: 0.4529
Epoch 21/100
110/110 [==============================] - ETA: 0s - loss: 0.9486 - accuracy: 0.6682
Epoch 21: val_loss improved from 1.15474 to 1.14608, saving model to best_custom_model.keras
1/1 [==============================] - 0s 28ms/step
 - val_f1_macro: 0.5745
110/110 [==============================] - 56s 505ms/step - loss: 0.9486 - accuracy: 0.6682 - val_loss: 1.1461 - val_accuracy: 0.6045 - lr: 5.0000e-05 - val_f1_macro: 0.5745
Epoch 22/100
110/110 [==============================] - ETA: 0s - loss: 0.9158 - accuracy: 0.6733
Epoch 22: val_loss did not improve from 1.14608
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.3960
110/110 [==============================] - 54s 491ms/step - loss: 0.9158 - accuracy: 0.6733 - val_loss: 1.4417 - val_accuracy: 0.5670 - lr: 2.5000e-05 - val_f1_macro: 0.3960
Epoch 23/100
110/110 [==============================] - ETA: 0s - loss: 0.8663 - accuracy: 0.7011
Epoch 23: val_loss improved from 1.14608 to 1.00274, saving model to best_custom_model.keras
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4466
110/110 [==============================] - 56s 505ms/step - loss: 0.8663 - accuracy: 0.7011 - val_loss: 1.0027 - val_accuracy: 0.6648 - lr: 1.2500e-05 - val_f1_macro: 0.4466
Epoch 24/100
110/110 [==============================] - ETA: 0s - loss: 0.8361 - accuracy: 0.7054
Epoch 24: val_loss improved from 1.00274 to 0.83762, saving model to best_custom_model.keras
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4977
110/110 [==============================] - 56s 504ms/step - loss: 0.8361 - accuracy: 0.7054 - val_loss: 0.8376 - val_accuracy: 0.6989 - lr: 6.2500e-06 - val_f1_macro: 0.4977
Epoch 25/100
110/110 [==============================] - ETA: 0s - loss: 0.8291 - accuracy: 0.7136
Epoch 25: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 29ms/step
 - val_f1_macro: 0.4812
110/110 [==============================] - 55s 495ms/step - loss: 0.8291 - accuracy: 0.7136 - val_loss: 0.8560 - val_accuracy: 0.6989 - lr: 3.1250e-06 - val_f1_macro: 0.4812
Epoch 26/100
110/110 [==============================] - ETA: 0s - loss: 0.8303 - accuracy: 0.7091
Epoch 26: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 491ms/step - loss: 0.8303 - accuracy: 0.7091 - val_loss: 0.8456 - val_accuracy: 0.7057 - lr: 1.5625e-06 - val_f1_macro: 0.4462
Epoch 27/100
110/110 [==============================] - ETA: 0s - loss: 0.8172 - accuracy: 0.7125
Epoch 27: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 29ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 493ms/step - loss: 0.8172 - accuracy: 0.7125 - val_loss: 0.8415 - val_accuracy: 0.7080 - lr: 7.8125e-07 - val_f1_macro: 0.4462
Epoch 28/100
110/110 [==============================] - ETA: 0s - loss: 0.8093 - accuracy: 0.7159
Epoch 28: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 32ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 494ms/step - loss: 0.8093 - accuracy: 0.7159 - val_loss: 0.8409 - val_accuracy: 0.7045 - lr: 3.9062e-07 - val_f1_macro: 0.4462
Epoch 29/100
110/110 [==============================] - ETA: 0s - loss: 0.8431 - accuracy: 0.7068
Epoch 29: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 495ms/step - loss: 0.8431 - accuracy: 0.7068 - val_loss: 0.8405 - val_accuracy: 0.7034 - lr: 1.9531e-07 - val_f1_macro: 0.4462
Epoch 30/100
110/110 [==============================] - ETA: 0s - loss: 0.8224 - accuracy: 0.7006
Epoch 30: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 497ms/step - loss: 0.8224 - accuracy: 0.7006 - val_loss: 0.8442 - val_accuracy: 0.7045 - lr: 9.7656e-08 - val_f1_macro: 0.4462
Epoch 31/100
110/110 [==============================] - ETA: 0s - loss: 0.8129 - accuracy: 0.7134
Epoch 31: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 498ms/step - loss: 0.8129 - accuracy: 0.7134 - val_loss: 0.8430 - val_accuracy: 0.7034 - lr: 4.8828e-08 - val_f1_macro: 0.4462
Epoch 32/100
110/110 [==============================] - ETA: 0s - loss: 0.8074 - accuracy: 0.7145
Epoch 32: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 500ms/step - loss: 0.8074 - accuracy: 0.7145 - val_loss: 0.8397 - val_accuracy: 0.7045 - lr: 4.8828e-09 - val_f1_macro: 0.4462
Epoch 33/100
110/110 [==============================] - ETA: 0s - loss: 0.8172 - accuracy: 0.7097
Epoch 33: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 495ms/step - loss: 0.8172 - accuracy: 0.7097 - val_loss: 0.8435 - val_accuracy: 0.7034 - lr: 4.8828e-10 - val_f1_macro: 0.4462
Epoch 34/100
110/110 [==============================] - ETA: 0s - loss: 0.8111 - accuracy: 0.7111
Epoch 34: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 499ms/step - loss: 0.8111 - accuracy: 0.7111 - val_loss: 0.8440 - val_accuracy: 0.7045 - lr: 4.8828e-11 - val_f1_macro: 0.4462
Epoch 35/100
110/110 [==============================] - ETA: 0s - loss: 0.8271 - accuracy: 0.7131
Epoch 35: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 498ms/step - loss: 0.8271 - accuracy: 0.7131 - val_loss: 0.8423 - val_accuracy: 0.7023 - lr: 4.8828e-12 - val_f1_macro: 0.4462
Epoch 36/100
110/110 [==============================] - ETA: 0s - loss: 0.8440 - accuracy: 0.7014
Epoch 36: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 491ms/step - loss: 0.8440 - accuracy: 0.7014 - val_loss: 0.8441 - val_accuracy: 0.7034 - lr: 4.8828e-13 - val_f1_macro: 0.4462
Epoch 37/100
110/110 [==============================] - ETA: 0s - loss: 0.7992 - accuracy: 0.7179
Epoch 37: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 493ms/step - loss: 0.7992 - accuracy: 0.7179 - val_loss: 0.8424 - val_accuracy: 0.7045 - lr: 4.8828e-14 - val_f1_macro: 0.4462
Epoch 38/100
110/110 [==============================] - ETA: 0s - loss: 0.8236 - accuracy: 0.7142
Epoch 38: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 492ms/step - loss: 0.8236 - accuracy: 0.7142 - val_loss: 0.8425 - val_accuracy: 0.7034 - lr: 4.8828e-15 - val_f1_macro: 0.4462
Epoch 39/100
110/110 [==============================] - ETA: 0s - loss: 0.8129 - accuracy: 0.7085
Epoch 39: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 28ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 492ms/step - loss: 0.8129 - accuracy: 0.7085 - val_loss: 0.8446 - val_accuracy: 0.7034 - lr: 4.8828e-16 - val_f1_macro: 0.4462
Epoch 40/100
110/110 [==============================] - ETA: 0s - loss: 0.8047 - accuracy: 0.7162
Epoch 40: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 494ms/step - loss: 0.8047 - accuracy: 0.7162 - val_loss: 0.8442 - val_accuracy: 0.7034 - lr: 4.8828e-17 - val_f1_macro: 0.4462
Epoch 41/100
110/110 [==============================] - ETA: 0s - loss: 0.7995 - accuracy: 0.7185
Epoch 41: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 495ms/step - loss: 0.7995 - accuracy: 0.7185 - val_loss: 0.8445 - val_accuracy: 0.7023 - lr: 4.8828e-18 - val_f1_macro: 0.4462
Epoch 42/100
110/110 [==============================] - ETA: 0s - loss: 0.8040 - accuracy: 0.7145
Epoch 42: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 495ms/step - loss: 0.8040 - accuracy: 0.7145 - val_loss: 0.8447 - val_accuracy: 0.7011 - lr: 4.8828e-19 - val_f1_macro: 0.4462
Epoch 43/100
110/110 [==============================] - ETA: 0s - loss: 0.8306 - accuracy: 0.7071
Epoch 43: val_loss did not improve from 0.83762
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 495ms/step - loss: 0.8306 - accuracy: 0.7071 - val_loss: 0.8403 - val_accuracy: 0.7034 - lr: 4.8828e-20 - val_f1_macro: 0.4462
Epoch 44/100
110/110 [==============================] - ETA: 0s - loss: 0.8231 - accuracy: 0.7082
Epoch 44: val_loss improved from 0.83762 to 0.83745, saving model to best_custom_model.keras
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 56s 508ms/step - loss: 0.8231 - accuracy: 0.7082 - val_loss: 0.8375 - val_accuracy: 0.7034 - lr: 4.8828e-21 - val_f1_macro: 0.4462
Epoch 45/100
110/110 [==============================] - ETA: 0s - loss: 0.8209 - accuracy: 0.7099
Epoch 45: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 32ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 492ms/step - loss: 0.8209 - accuracy: 0.7099 - val_loss: 0.8424 - val_accuracy: 0.7023 - lr: 4.8828e-22 - val_f1_macro: 0.4462
Epoch 46/100
110/110 [==============================] - ETA: 0s - loss: 0.8280 - accuracy: 0.7065
Epoch 46: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 493ms/step - loss: 0.8280 - accuracy: 0.7065 - val_loss: 0.8433 - val_accuracy: 0.7057 - lr: 4.8828e-23 - val_f1_macro: 0.4462
Epoch 47/100
110/110 [==============================] - ETA: 0s - loss: 0.8224 - accuracy: 0.7094
Epoch 47: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 493ms/step - loss: 0.8224 - accuracy: 0.7094 - val_loss: 0.8425 - val_accuracy: 0.7057 - lr: 4.8828e-24 - val_f1_macro: 0.4462
Epoch 48/100
110/110 [==============================] - ETA: 0s - loss: 0.8321 - accuracy: 0.7034
Epoch 48: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 29ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 496ms/step - loss: 0.8321 - accuracy: 0.7034 - val_loss: 0.8424 - val_accuracy: 0.7023 - lr: 4.8828e-25 - val_f1_macro: 0.4462
Epoch 49/100
110/110 [==============================] - ETA: 0s - loss: 0.8102 - accuracy: 0.7139
Epoch 49: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 496ms/step - loss: 0.8102 - accuracy: 0.7139 - val_loss: 0.8419 - val_accuracy: 0.7045 - lr: 4.8828e-26 - val_f1_macro: 0.4462
Epoch 50/100
110/110 [==============================] - ETA: 0s - loss: 0.8184 - accuracy: 0.7145
Epoch 50: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 29ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 496ms/step - loss: 0.8184 - accuracy: 0.7145 - val_loss: 0.8445 - val_accuracy: 0.7045 - lr: 4.8828e-27 - val_f1_macro: 0.4462
Epoch 51/100
110/110 [==============================] - ETA: 0s - loss: 0.8076 - accuracy: 0.7227
Epoch 51: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 500ms/step - loss: 0.8076 - accuracy: 0.7227 - val_loss: 0.8419 - val_accuracy: 0.7034 - lr: 4.8828e-28 - val_f1_macro: 0.4462
Epoch 52/100
110/110 [==============================] - ETA: 0s - loss: 0.8269 - accuracy: 0.7077
Epoch 52: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 498ms/step - loss: 0.8269 - accuracy: 0.7077 - val_loss: 0.8413 - val_accuracy: 0.7034 - lr: 4.8828e-29 - val_f1_macro: 0.4462
Epoch 53/100
110/110 [==============================] - ETA: 0s - loss: 0.8206 - accuracy: 0.7134
Epoch 53: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 497ms/step - loss: 0.8206 - accuracy: 0.7134 - val_loss: 0.8403 - val_accuracy: 0.7068 - lr: 4.8828e-30 - val_f1_macro: 0.4462
Epoch 54/100
110/110 [==============================] - ETA: 0s - loss: 0.8166 - accuracy: 0.7247
Epoch 54: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 497ms/step - loss: 0.8166 - accuracy: 0.7247 - val_loss: 0.8414 - val_accuracy: 0.7034 - lr: 4.8828e-31 - val_f1_macro: 0.4462
Epoch 55/100
110/110 [==============================] - ETA: 0s - loss: 0.8324 - accuracy: 0.7057
Epoch 55: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 32ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 489ms/step - loss: 0.8324 - accuracy: 0.7057 - val_loss: 0.8419 - val_accuracy: 0.7045 - lr: 4.8828e-32 - val_f1_macro: 0.4462
Epoch 56/100
110/110 [==============================] - ETA: 0s - loss: 0.8282 - accuracy: 0.7097
Epoch 56: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 28ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 490ms/step - loss: 0.8282 - accuracy: 0.7097 - val_loss: 0.8430 - val_accuracy: 0.7045 - lr: 4.8828e-33 - val_f1_macro: 0.4462
Epoch 57/100
110/110 [==============================] - ETA: 0s - loss: 0.8172 - accuracy: 0.7151
Epoch 57: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 494ms/step - loss: 0.8172 - accuracy: 0.7151 - val_loss: 0.8384 - val_accuracy: 0.7057 - lr: 4.8828e-34 - val_f1_macro: 0.4462
Epoch 58/100
110/110 [==============================] - ETA: 0s - loss: 0.8134 - accuracy: 0.7071
Epoch 58: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 492ms/step - loss: 0.8134 - accuracy: 0.7071 - val_loss: 0.8422 - val_accuracy: 0.7057 - lr: 4.8828e-35 - val_f1_macro: 0.4462
Epoch 59/100
110/110 [==============================] - ETA: 0s - loss: 0.8351 - accuracy: 0.7094
Epoch 59: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 492ms/step - loss: 0.8351 - accuracy: 0.7094 - val_loss: 0.8383 - val_accuracy: 0.7080 - lr: 4.8828e-36 - val_f1_macro: 0.4462
Epoch 60/100
110/110 [==============================] - ETA: 0s - loss: 0.8135 - accuracy: 0.7094
Epoch 60: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 493ms/step - loss: 0.8135 - accuracy: 0.7094 - val_loss: 0.8397 - val_accuracy: 0.7045 - lr: 4.8828e-37 - val_f1_macro: 0.4462
Epoch 61/100
110/110 [==============================] - ETA: 0s - loss: 0.8036 - accuracy: 0.7173
Epoch 61: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 494ms/step - loss: 0.8036 - accuracy: 0.7173 - val_loss: 0.8397 - val_accuracy: 0.7057 - lr: 4.8828e-38 - val_f1_macro: 0.4462
Epoch 62/100
110/110 [==============================] - ETA: 0s - loss: 0.8104 - accuracy: 0.7097
Epoch 62: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 497ms/step - loss: 0.8104 - accuracy: 0.7097 - val_loss: 0.8424 - val_accuracy: 0.7045 - lr: 4.8828e-39 - val_f1_macro: 0.4462
Epoch 63/100
110/110 [==============================] - ETA: 0s - loss: 0.8196 - accuracy: 0.7182
Epoch 63: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 497ms/step - loss: 0.8196 - accuracy: 0.7182 - val_loss: 0.8403 - val_accuracy: 0.7034 - lr: 4.8828e-40 - val_f1_macro: 0.4462
Epoch 64/100
110/110 [==============================] - ETA: 0s - loss: 0.7952 - accuracy: 0.7185
Epoch 64: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 31ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 499ms/step - loss: 0.7952 - accuracy: 0.7185 - val_loss: 0.8436 - val_accuracy: 0.7034 - lr: 4.8828e-41 - val_f1_macro: 0.4462
Epoch 65/100
110/110 [==============================] - ETA: 0s - loss: 0.8394 - accuracy: 0.7068
Epoch 65: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 30ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 490ms/step - loss: 0.8394 - accuracy: 0.7068 - val_loss: 0.8463 - val_accuracy: 0.7045 - lr: 4.8821e-42 - val_f1_macro: 0.4462
Epoch 66/100
110/110 [==============================] - ETA: 0s - loss: 0.8303 - accuracy: 0.7051
Epoch 66: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 491ms/step - loss: 0.8303 - accuracy: 0.7051 - val_loss: 0.8458 - val_accuracy: 0.7045 - lr: 4.8765e-43 - val_f1_macro: 0.4462
Epoch 67/100
110/110 [==============================] - ETA: 0s - loss: 0.8391 - accuracy: 0.7080
Epoch 67: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 29ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 54s 493ms/step - loss: 0.8391 - accuracy: 0.7080 - val_loss: 0.8456 - val_accuracy: 0.7045 - lr: 4.9045e-44 - val_f1_macro: 0.4462
Epoch 68/100
110/110 [==============================] - ETA: 0s - loss: 0.7931 - accuracy: 0.7273
Epoch 68: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 29ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 496ms/step - loss: 0.7931 - accuracy: 0.7273 - val_loss: 0.8419 - val_accuracy: 0.7068 - lr: 5.6052e-45 - val_f1_macro: 0.4462
Epoch 69/100
110/110 [==============================] - ETA: 0s - loss: 0.8193 - accuracy: 0.7097Restoring model weights from the end of the best epoch: 44.

Epoch 69: val_loss did not improve from 0.83745
1/1 [==============================] - 0s 27ms/step
 - val_f1_macro: 0.4462
110/110 [==============================] - 55s 494ms/step - loss: 0.8193 - accuracy: 0.7097 - val_loss: 0.8429 - val_accuracy: 0.7034 - lr: 0.0000e+00 - val_f1_macro: 0.4462
Epoch 69: early stopping
35/35 [==============================] - 5s 143ms/step

Классификационный отчет:
              precision    recall  f1-score   support

       Beige       0.68      0.72      0.70       100
       Black       0.48      0.70      0.57       100
        Blue       0.70      0.74      0.72       100
       Brown       0.64      0.58      0.61       100
       Green       0.73      0.63      0.68       100
        Grey       0.46      0.53      0.49       100
      Orange       0.95      0.75      0.84       100
         Red       0.84      0.94      0.89       100
      Silver       0.58      0.56      0.57       100
       White       0.95      0.63      0.76       100
      Yellow       0.97      0.95      0.96       100

    accuracy                           0.76       910
   macro avg       0.77      0.74      0.75       910
weighted avg       0.78      0.76      0.76       910


F1 Macro: 0.7454
"""

## Итоги по первому эксперименту
### F1 Macro: 0.7454
![Графики обучения модели](plots/graph_custom.jpg)
![Матрица ошибок](plots/matrix_custom.jpg)


## Вывод для второго эксперимента
Вывод скопировал из консоли домашнего компьютера, тк там не запустился jupiter notebook, а на ноутбуке было грустно из-за отсутствия видеокарты


In [ ]:
"""
=== Эксперимент с предобученной моделью InceptionV3 ===
Режим обучения: GPU
Распределение цветов в исходном датасете:
color
Black          14317
Grey            9474
White           9395
Blue            8483
Silver          7770
Red             6095
Unlisted        1516
Brown            911
Green            777
Yellow           667
Beige            600
Orange           559
Purple           362
Bronze           329
Gold             217
Multicolour      196
Pink              87
Turquoise         26
Maroon            26
Burgundy           9
Magenta            9
Navy               1
Indigo             1
Name: count, dtype: int64
Количество классов: 23
Распределение цветов в отфильтрованном и нормализованном датасете:
color
Black     500
Grey      500
White     500
Blue      500
Silver    500
Red       500
Brown     500
Green     500
Yellow    500
Beige     500
Orange    500
Name: count, dtype: int64
Количество классов: 11
Размер обучающей выборки: 3520
Размер валидационной выборки: 880
Размер тестовой выборки: 1100
Found 3520 validated image filenames belonging to 11 classes.
Found 880 validated image filenames belonging to 11 classes.
Found 1100 validated image filenames belonging to 11 classes.
Найдено 1 GPU устройств. Будет использоваться GPU.
Epoch 1/100
110/110 [==============================] - ETA: 0s - loss: 2.3727 - accuracy: 0.1489
Epoch 1: val_loss improved from inf to 2.13398, saving model to best_model.keras
110/110 [==============================] - 60s 499ms/step - loss: 2.3727 - accuracy: 0.1489 - val_loss: 2.1340 - val_accuracy: 0.3102 - lr: 5.0000e-05
Epoch 2/100
110/110 [==============================] - ETA: 0s - loss: 2.1055 - accuracy: 0.2497
Epoch 2: val_loss improved from 2.13398 to 1.87985, saving model to best_model.keras
110/110 [==============================] - 54s 493ms/step - loss: 2.1055 - accuracy: 0.2497 - val_loss: 1.8798 - val_accuracy: 0.3739 - lr: 5.0000e-05
Epoch 3/100
110/110 [==============================] - ETA: 0s - loss: 1.8987 - accuracy: 0.3224
Epoch 3: val_loss improved from 1.87985 to 1.71647, saving model to best_model.keras
110/110 [==============================] - 54s 494ms/step - loss: 1.8987 - accuracy: 0.3224 - val_loss: 1.7165 - val_accuracy: 0.4432 - lr: 5.0000e-05
Epoch 4/100
110/110 [==============================] - ETA: 0s - loss: 1.7894 - accuracy: 0.3438
Epoch 4: val_loss improved from 1.71647 to 1.62447, saving model to best_model.keras
110/110 [==============================] - 54s 494ms/step - loss: 1.7894 - accuracy: 0.3438 - val_loss: 1.6245 - val_accuracy: 0.4455 - lr: 5.0000e-05
Epoch 5/100
110/110 [==============================] - ETA: 0s - loss: 1.6949 - accuracy: 0.3793
Epoch 5: val_loss improved from 1.62447 to 1.54841, saving model to best_model.keras
110/110 [==============================] - 55s 497ms/step - loss: 1.6949 - accuracy: 0.3793 - val_loss: 1.5484 - val_accuracy: 0.4875 - lr: 5.0000e-05
Epoch 6/100
110/110 [==============================] - ETA: 0s - loss: 1.6302 - accuracy: 0.4023
Epoch 6: val_loss improved from 1.54841 to 1.51023, saving model to best_model.keras
110/110 [==============================] - 55s 503ms/step - loss: 1.6302 - accuracy: 0.4023 - val_loss: 1.5102 - val_accuracy: 0.4682 - lr: 5.0000e-05
Epoch 7/100
110/110 [==============================] - ETA: 0s - loss: 1.5634 - accuracy: 0.4227
Epoch 7: val_loss improved from 1.51023 to 1.45108, saving model to best_model.keras
110/110 [==============================] - 54s 492ms/step - loss: 1.5634 - accuracy: 0.4227 - val_loss: 1.4511 - val_accuracy: 0.4943 - lr: 5.0000e-05
Epoch 8/100
110/110 [==============================] - ETA: 0s - loss: 1.5359 - accuracy: 0.4352
Epoch 8: val_loss improved from 1.45108 to 1.40813, saving model to best_model.keras
110/110 [==============================] - 56s 504ms/step - loss: 1.5359 - accuracy: 0.4352 - val_loss: 1.4081 - val_accuracy: 0.5170 - lr: 5.0000e-05
Epoch 9/100
110/110 [==============================] - ETA: 0s - loss: 1.4929 - accuracy: 0.4517
Epoch 9: val_loss improved from 1.40813 to 1.37302, saving model to best_model.keras
110/110 [==============================] - 55s 495ms/step - loss: 1.4929 - accuracy: 0.4517 - val_loss: 1.3730 - val_accuracy: 0.5318 - lr: 5.0000e-05
Epoch 10/100
110/110 [==============================] - ETA: 0s - loss: 1.4761 - accuracy: 0.4645
Epoch 10: val_loss improved from 1.37302 to 1.34285, saving model to best_model.keras
110/110 [==============================] - 53s 484ms/step - loss: 1.4761 - accuracy: 0.4645 - val_loss: 1.3429 - val_accuracy: 0.5455 - lr: 5.0000e-05
Epoch 11/100
110/110 [==============================] - ETA: 0s - loss: 1.4227 - accuracy: 0.4719
Epoch 11: val_loss improved from 1.34285 to 1.33842, saving model to best_model.keras
110/110 [==============================] - 53s 483ms/step - loss: 1.4227 - accuracy: 0.4719 - val_loss: 1.3384 - val_accuracy: 0.5386 - lr: 5.0000e-05
Epoch 12/100
110/110 [==============================] - ETA: 0s - loss: 1.3935 - accuracy: 0.4852
Epoch 12: val_loss improved from 1.33842 to 1.29965, saving model to best_model.keras
110/110 [==============================] - 54s 492ms/step - loss: 1.3935 - accuracy: 0.4852 - val_loss: 1.2996 - val_accuracy: 0.5489 - lr: 5.0000e-05
Epoch 13/100
110/110 [==============================] - ETA: 0s - loss: 1.3864 - accuracy: 0.5014
Epoch 13: val_loss improved from 1.29965 to 1.27937, saving model to best_model.keras
110/110 [==============================] - 54s 492ms/step - loss: 1.3864 - accuracy: 0.5014 - val_loss: 1.2794 - val_accuracy: 0.5705 - lr: 5.0000e-05
Epoch 14/100
110/110 [==============================] - ETA: 0s - loss: 1.3739 - accuracy: 0.4929
Epoch 14: val_loss did not improve from 1.27937
110/110 [==============================] - 53s 481ms/step - loss: 1.3739 - accuracy: 0.4929 - val_loss: 1.2838 - val_accuracy: 0.5591 - lr: 5.0000e-05
Epoch 15/100
110/110 [==============================] - ETA: 0s - loss: 1.3388 - accuracy: 0.5105
Epoch 15: val_loss improved from 1.27937 to 1.25111, saving model to best_model.keras
110/110 [==============================] - 54s 488ms/step - loss: 1.3388 - accuracy: 0.5105 - val_loss: 1.2511 - val_accuracy: 0.5705 - lr: 5.0000e-05
Epoch 16/100
110/110 [==============================] - ETA: 0s - loss: 1.3141 - accuracy: 0.5182
Epoch 16: val_loss improved from 1.25111 to 1.24000, saving model to best_model.keras
110/110 [==============================] - 53s 481ms/step - loss: 1.3141 - accuracy: 0.5182 - val_loss: 1.2400 - val_accuracy: 0.5602 - lr: 5.0000e-05
Epoch 17/100
110/110 [==============================] - ETA: 0s - loss: 1.2989 - accuracy: 0.5310
Epoch 17: val_loss improved from 1.24000 to 1.22536, saving model to best_model.keras
110/110 [==============================] - 54s 492ms/step - loss: 1.2989 - accuracy: 0.5310 - val_loss: 1.2254 - val_accuracy: 0.5648 - lr: 5.0000e-05
Epoch 18/100
110/110 [==============================] - ETA: 0s - loss: 1.2903 - accuracy: 0.5324
Epoch 18: val_loss improved from 1.22536 to 1.20828, saving model to best_model.keras
110/110 [==============================] - 53s 484ms/step - loss: 1.2903 - accuracy: 0.5324 - val_loss: 1.2083 - val_accuracy: 0.5818 - lr: 5.0000e-05
Epoch 19/100
110/110 [==============================] - ETA: 0s - loss: 1.2646 - accuracy: 0.5403
Epoch 19: val_loss improved from 1.20828 to 1.19477, saving model to best_model.keras
110/110 [==============================] - 53s 482ms/step - loss: 1.2646 - accuracy: 0.5403 - val_loss: 1.1948 - val_accuracy: 0.5807 - lr: 5.0000e-05
Epoch 20/100
110/110 [==============================] - ETA: 0s - loss: 1.2652 - accuracy: 0.5415
Epoch 20: val_loss did not improve from 1.19477
110/110 [==============================] - 52s 470ms/step - loss: 1.2652 - accuracy: 0.5415 - val_loss: 1.2077 - val_accuracy: 0.5773 - lr: 5.0000e-05
Epoch 21/100
110/110 [==============================] - ETA: 0s - loss: 1.2259 - accuracy: 0.5528
Epoch 21: val_loss improved from 1.19477 to 1.17525, saving model to best_model.keras
110/110 [==============================] - 53s 482ms/step - loss: 1.2259 - accuracy: 0.5528 - val_loss: 1.1752 - val_accuracy: 0.5830 - lr: 5.0000e-05
Epoch 22/100
110/110 [==============================] - ETA: 0s - loss: 1.2047 - accuracy: 0.5611
Epoch 22: val_loss improved from 1.17525 to 1.16405, saving model to best_model.keras
110/110 [==============================] - 53s 483ms/step - loss: 1.2047 - accuracy: 0.5611 - val_loss: 1.1640 - val_accuracy: 0.5852 - lr: 2.5000e-05
Epoch 23/100
110/110 [==============================] - ETA: 0s - loss: 1.1853 - accuracy: 0.5688
Epoch 23: val_loss improved from 1.16405 to 1.14050, saving model to best_model.keras
110/110 [==============================] - 53s 482ms/step - loss: 1.1853 - accuracy: 0.5688 - val_loss: 1.1405 - val_accuracy: 0.6091 - lr: 1.2500e-05
Epoch 24/100
110/110 [==============================] - ETA: 0s - loss: 1.1792 - accuracy: 0.5750
Epoch 24: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 470ms/step - loss: 1.1792 - accuracy: 0.5750 - val_loss: 1.1462 - val_accuracy: 0.6000 - lr: 6.2500e-06
Epoch 25/100
110/110 [==============================] - ETA: 0s - loss: 1.1748 - accuracy: 0.5651
Epoch 25: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 471ms/step - loss: 1.1748 - accuracy: 0.5651 - val_loss: 1.1411 - val_accuracy: 0.6034 - lr: 3.1250e-06
Epoch 26/100
110/110 [==============================] - ETA: 0s - loss: 1.1662 - accuracy: 0.5815
Epoch 26: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 470ms/step - loss: 1.1662 - accuracy: 0.5815 - val_loss: 1.1410 - val_accuracy: 0.6068 - lr: 1.5625e-06
Epoch 27/100
110/110 [==============================] - ETA: 0s - loss: 1.1542 - accuracy: 0.5892
Epoch 27: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 469ms/step - loss: 1.1542 - accuracy: 0.5892 - val_loss: 1.1408 - val_accuracy: 0.6068 - lr: 7.8125e-07
Epoch 28/100
110/110 [==============================] - ETA: 0s - loss: 1.1823 - accuracy: 0.5730
Epoch 28: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 471ms/step - loss: 1.1823 - accuracy: 0.5730 - val_loss: 1.1407 - val_accuracy: 0.6068 - lr: 3.9062e-07
Epoch 29/100
110/110 [==============================] - ETA: 0s - loss: 1.1532 - accuracy: 0.5884
Epoch 29: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 471ms/step - loss: 1.1532 - accuracy: 0.5884 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 1.9531e-07
Epoch 30/100
110/110 [==============================] - ETA: 0s - loss: 1.1709 - accuracy: 0.5707
Epoch 30: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 471ms/step - loss: 1.1709 - accuracy: 0.5707 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 9.7656e-08
Epoch 31/100
110/110 [==============================] - ETA: 0s - loss: 1.1611 - accuracy: 0.5855
Epoch 31: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 470ms/step - loss: 1.1611 - accuracy: 0.5855 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-08
Epoch 32/100
110/110 [==============================] - ETA: 0s - loss: 1.1577 - accuracy: 0.5744
Epoch 32: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 471ms/step - loss: 1.1577 - accuracy: 0.5744 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-09
Epoch 33/100
110/110 [==============================] - ETA: 0s - loss: 1.1468 - accuracy: 0.5750
Epoch 33: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 469ms/step - loss: 1.1468 - accuracy: 0.5750 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-10
Epoch 34/100
110/110 [==============================] - ETA: 0s - loss: 1.1601 - accuracy: 0.5722
Epoch 34: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 470ms/step - loss: 1.1601 - accuracy: 0.5722 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-11
Epoch 35/100
110/110 [==============================] - ETA: 0s - loss: 1.1576 - accuracy: 0.5895
Epoch 35: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 470ms/step - loss: 1.1576 - accuracy: 0.5895 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-12
Epoch 36/100
110/110 [==============================] - ETA: 0s - loss: 1.1713 - accuracy: 0.5690
Epoch 36: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 470ms/step - loss: 1.1713 - accuracy: 0.5690 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-13
Epoch 37/100
110/110 [==============================] - ETA: 0s - loss: 1.1679 - accuracy: 0.5781
Epoch 37: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 470ms/step - loss: 1.1679 - accuracy: 0.5781 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-14
Epoch 38/100
110/110 [==============================] - ETA: 0s - loss: 1.1729 - accuracy: 0.5801
Epoch 38: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 471ms/step - loss: 1.1729 - accuracy: 0.5801 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-15
Epoch 39/100
110/110 [==============================] - ETA: 0s - loss: 1.1713 - accuracy: 0.5741
Epoch 39: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 471ms/step - loss: 1.1713 - accuracy: 0.5741 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-16
Epoch 40/100
110/110 [==============================] - ETA: 0s - loss: 1.1726 - accuracy: 0.5793
Epoch 40: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 470ms/step - loss: 1.1726 - accuracy: 0.5793 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-17
Epoch 41/100
110/110 [==============================] - ETA: 0s - loss: 1.1702 - accuracy: 0.5770
Epoch 41: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 472ms/step - loss: 1.1702 - accuracy: 0.5770 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-18
Epoch 42/100
110/110 [==============================] - ETA: 0s - loss: 1.1700 - accuracy: 0.5750
Epoch 42: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 471ms/step - loss: 1.1700 - accuracy: 0.5750 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-19
Epoch 43/100
110/110 [==============================] - ETA: 0s - loss: 1.1763 - accuracy: 0.5736Restoring model weights from the end of the best epoch: 23.

Epoch 43: val_loss did not improve from 1.14050
110/110 [==============================] - 52s 472ms/step - loss: 1.1763 - accuracy: 0.5736 - val_loss: 1.1406 - val_accuracy: 0.6068 - lr: 4.8828e-20
Epoch 43: early stopping

Дообучение с разблокировкой последних слоев...
Epoch 1/100
110/110 [==============================] - ETA: 0s - loss: 1.2504 - accuracy: 0.5384
Epoch 1: val_loss did not improve from 1.14050
110/110 [==============================] - 57s 487ms/step - loss: 1.2504 - accuracy: 0.5384 - val_loss: 1.1454 - val_accuracy: 0.6034 - lr: 5.0000e-06
Epoch 2/100
110/110 [==============================] - ETA: 0s - loss: 1.2074 - accuracy: 0.5639
Epoch 2: val_loss improved from 1.14050 to 1.13691, saving model to best_model.keras
110/110 [==============================] - 55s 496ms/step - loss: 1.2074 - accuracy: 0.5639 - val_loss: 1.1369 - val_accuracy: 0.6000 - lr: 5.0000e-06
Epoch 3/100
110/110 [==============================] - ETA: 0s - loss: 1.1969 - accuracy: 0.5653
Epoch 3: val_loss improved from 1.13691 to 1.13140, saving model to best_model.keras
110/110 [==============================] - 54s 491ms/step - loss: 1.1969 - accuracy: 0.5653 - val_loss: 1.1314 - val_accuracy: 0.6091 - lr: 5.0000e-06
Epoch 4/100
110/110 [==============================] - ETA: 0s - loss: 1.1840 - accuracy: 0.5753
Epoch 4: val_loss improved from 1.13140 to 1.12736, saving model to best_model.keras
110/110 [==============================] - 55s 496ms/step - loss: 1.1840 - accuracy: 0.5753 - val_loss: 1.1274 - val_accuracy: 0.6114 - lr: 5.0000e-06
Epoch 5/100
110/110 [==============================] - ETA: 0s - loss: 1.1584 - accuracy: 0.5787
Epoch 5: val_loss improved from 1.12736 to 1.12064, saving model to best_model.keras
110/110 [==============================] - 68s 617ms/step - loss: 1.1584 - accuracy: 0.5787 - val_loss: 1.1206 - val_accuracy: 0.6216 - lr: 5.0000e-06
Epoch 6/100
110/110 [==============================] - ETA: 0s - loss: 1.1644 - accuracy: 0.5838
Epoch 6: val_loss improved from 1.12064 to 1.11523, saving model to best_model.keras
110/110 [==============================] - 64s 581ms/step - loss: 1.1644 - accuracy: 0.5838 - val_loss: 1.1152 - val_accuracy: 0.6205 - lr: 5.0000e-06
Epoch 7/100
110/110 [==============================] - ETA: 0s - loss: 1.1677 - accuracy: 0.5753
Epoch 7: val_loss improved from 1.11523 to 1.11453, saving model to best_model.keras
110/110 [==============================] - 55s 494ms/step - loss: 1.1677 - accuracy: 0.5753 - val_loss: 1.1145 - val_accuracy: 0.6125 - lr: 5.0000e-06
Epoch 8/100
110/110 [==============================] - ETA: 0s - loss: 1.1351 - accuracy: 0.5849
Epoch 8: val_loss improved from 1.11453 to 1.10784, saving model to best_model.keras
110/110 [==============================] - 55s 493ms/step - loss: 1.1351 - accuracy: 0.5849 - val_loss: 1.1078 - val_accuracy: 0.6205 - lr: 5.0000e-06
Epoch 9/100
110/110 [==============================] - ETA: 0s - loss: 1.1424 - accuracy: 0.5875
Epoch 9: val_loss improved from 1.10784 to 1.10109, saving model to best_model.keras
110/110 [==============================] - 54s 488ms/step - loss: 1.1424 - accuracy: 0.5875 - val_loss: 1.1011 - val_accuracy: 0.6239 - lr: 5.0000e-06
Epoch 10/100
110/110 [==============================] - ETA: 0s - loss: 1.1386 - accuracy: 0.5872
Epoch 10: val_loss did not improve from 1.10109
110/110 [==============================] - 52s 474ms/step - loss: 1.1386 - accuracy: 0.5872 - val_loss: 1.1020 - val_accuracy: 0.6227 - lr: 5.0000e-06
Epoch 11/100
110/110 [==============================] - ETA: 0s - loss: 1.1093 - accuracy: 0.5980
Epoch 11: val_loss improved from 1.10109 to 1.09721, saving model to best_model.keras
110/110 [==============================] - 54s 494ms/step - loss: 1.1093 - accuracy: 0.5980 - val_loss: 1.0972 - val_accuracy: 0.6216 - lr: 5.0000e-06
Epoch 12/100
110/110 [==============================] - ETA: 0s - loss: 1.1269 - accuracy: 0.5983
Epoch 12: val_loss improved from 1.09721 to 1.09321, saving model to best_model.keras
110/110 [==============================] - 54s 486ms/step - loss: 1.1269 - accuracy: 0.5983 - val_loss: 1.0932 - val_accuracy: 0.6250 - lr: 5.0000e-06
Epoch 13/100
110/110 [==============================] - ETA: 0s - loss: 1.1249 - accuracy: 0.5858
Epoch 13: val_loss improved from 1.09321 to 1.09287, saving model to best_model.keras
110/110 [==============================] - 54s 489ms/step - loss: 1.1249 - accuracy: 0.5858 - val_loss: 1.0929 - val_accuracy: 0.6148 - lr: 5.0000e-06
Epoch 14/100
110/110 [==============================] - ETA: 0s - loss: 1.1166 - accuracy: 0.6000
Epoch 14: val_loss improved from 1.09287 to 1.08694, saving model to best_model.keras
110/110 [==============================] - 55s 497ms/step - loss: 1.1166 - accuracy: 0.6000 - val_loss: 1.0869 - val_accuracy: 0.6250 - lr: 5.0000e-06
Epoch 15/100
110/110 [==============================] - ETA: 0s - loss: 1.1221 - accuracy: 0.6028
Epoch 15: val_loss improved from 1.08694 to 1.08551, saving model to best_model.keras
110/110 [==============================] - 54s 488ms/step - loss: 1.1221 - accuracy: 0.6028 - val_loss: 1.0855 - val_accuracy: 0.6250 - lr: 5.0000e-06
Epoch 16/100
110/110 [==============================] - ETA: 0s - loss: 1.1106 - accuracy: 0.5989
Epoch 16: val_loss improved from 1.08551 to 1.08033, saving model to best_model.keras
110/110 [==============================] - 54s 488ms/step - loss: 1.1106 - accuracy: 0.5989 - val_loss: 1.0803 - val_accuracy: 0.6261 - lr: 5.0000e-06
Epoch 17/100
110/110 [==============================] - ETA: 0s - loss: 1.0777 - accuracy: 0.6136
Epoch 17: val_loss improved from 1.08033 to 1.07700, saving model to best_model.keras
110/110 [==============================] - 54s 489ms/step - loss: 1.0777 - accuracy: 0.6136 - val_loss: 1.0770 - val_accuracy: 0.6261 - lr: 5.0000e-06
Epoch 18/100
110/110 [==============================] - ETA: 0s - loss: 1.1008 - accuracy: 0.6020
Epoch 18: val_loss improved from 1.07700 to 1.07658, saving model to best_model.keras
110/110 [==============================] - 54s 489ms/step - loss: 1.1008 - accuracy: 0.6020 - val_loss: 1.0766 - val_accuracy: 0.6216 - lr: 5.0000e-06
Epoch 19/100
110/110 [==============================] - ETA: 0s - loss: 1.0871 - accuracy: 0.6088
Epoch 19: val_loss improved from 1.07658 to 1.07266, saving model to best_model.keras
110/110 [==============================] - 54s 487ms/step - loss: 1.0871 - accuracy: 0.6088 - val_loss: 1.0727 - val_accuracy: 0.6159 - lr: 5.0000e-06
Epoch 20/100
110/110 [==============================] - ETA: 0s - loss: 1.0711 - accuracy: 0.6210
Epoch 20: val_loss did not improve from 1.07266
110/110 [==============================] - 52s 474ms/step - loss: 1.0711 - accuracy: 0.6210 - val_loss: 1.0735 - val_accuracy: 0.6205 - lr: 5.0000e-06
Epoch 21/100
110/110 [==============================] - ETA: 0s - loss: 1.0753 - accuracy: 0.6088
Epoch 21: val_loss improved from 1.07266 to 1.07042, saving model to best_model.keras
110/110 [==============================] - 55s 496ms/step - loss: 1.0753 - accuracy: 0.6088 - val_loss: 1.0704 - val_accuracy: 0.6205 - lr: 5.0000e-06
Epoch 22/100
110/110 [==============================] - ETA: 0s - loss: 1.0760 - accuracy: 0.6108
Epoch 22: val_loss improved from 1.07042 to 1.06485, saving model to best_model.keras
110/110 [==============================] - 55s 494ms/step - loss: 1.0760 - accuracy: 0.6108 - val_loss: 1.0649 - val_accuracy: 0.6261 - lr: 2.5000e-06
Epoch 23/100
110/110 [==============================] - ETA: 0s - loss: 1.0718 - accuracy: 0.6108
Epoch 23: val_loss improved from 1.06485 to 1.06448, saving model to best_model.keras
110/110 [==============================] - 54s 493ms/step - loss: 1.0718 - accuracy: 0.6108 - val_loss: 1.0645 - val_accuracy: 0.6261 - lr: 1.2500e-06
Epoch 24/100
110/110 [==============================] - ETA: 0s - loss: 1.0816 - accuracy: 0.6159
Epoch 24: val_loss did not improve from 1.06448
110/110 [==============================] - 52s 474ms/step - loss: 1.0816 - accuracy: 0.6159 - val_loss: 1.0645 - val_accuracy: 0.6250 - lr: 6.2500e-07
Epoch 25/100
110/110 [==============================] - ETA: 0s - loss: 1.0503 - accuracy: 0.6227
Epoch 25: val_loss improved from 1.06448 to 1.06377, saving model to best_model.keras
110/110 [==============================] - 54s 489ms/step - loss: 1.0503 - accuracy: 0.6227 - val_loss: 1.0638 - val_accuracy: 0.6261 - lr: 3.1250e-07
Epoch 26/100
110/110 [==============================] - ETA: 0s - loss: 1.0880 - accuracy: 0.6051
Epoch 26: val_loss did not improve from 1.06377
110/110 [==============================] - 52s 473ms/step - loss: 1.0880 - accuracy: 0.6051 - val_loss: 1.0640 - val_accuracy: 0.6250 - lr: 1.5625e-07
Epoch 27/100
110/110 [==============================] - ETA: 0s - loss: 1.0708 - accuracy: 0.6128
Epoch 27: val_loss did not improve from 1.06377
110/110 [==============================] - 52s 474ms/step - loss: 1.0708 - accuracy: 0.6128 - val_loss: 1.0641 - val_accuracy: 0.6250 - lr: 7.8125e-08
Epoch 28/100
110/110 [==============================] - ETA: 0s - loss: 1.0703 - accuracy: 0.6153
Epoch 28: val_loss did not improve from 1.06377
110/110 [==============================] - 52s 474ms/step - loss: 1.0703 - accuracy: 0.6153 - val_loss: 1.0642 - val_accuracy: 0.6239 - lr: 3.9062e-08
Epoch 29/100
110/110 [==============================] - ETA: 0s - loss: 1.0898 - accuracy: 0.5983
Epoch 29: val_loss improved from 1.06377 to 1.06351, saving model to best_model.keras
110/110 [==============================] - 54s 487ms/step - loss: 1.0898 - accuracy: 0.5983 - val_loss: 1.0635 - val_accuracy: 0.6261 - lr: 1.9531e-08
Epoch 30/100
110/110 [==============================] - ETA: 0s - loss: 1.0940 - accuracy: 0.5949
Epoch 30: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 475ms/step - loss: 1.0940 - accuracy: 0.5949 - val_loss: 1.0636 - val_accuracy: 0.6261 - lr: 9.7656e-09
Epoch 31/100
110/110 [==============================] - ETA: 0s - loss: 1.0671 - accuracy: 0.6244
Epoch 31: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0671 - accuracy: 0.6244 - val_loss: 1.0641 - val_accuracy: 0.6261 - lr: 4.8828e-09
Epoch 32/100
110/110 [==============================] - ETA: 0s - loss: 1.0611 - accuracy: 0.6179
Epoch 32: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0611 - accuracy: 0.6179 - val_loss: 1.0644 - val_accuracy: 0.6227 - lr: 4.8828e-10
Epoch 33/100
110/110 [==============================] - ETA: 0s - loss: 1.0829 - accuracy: 0.6122
Epoch 33: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0829 - accuracy: 0.6122 - val_loss: 1.0638 - val_accuracy: 0.6261 - lr: 4.8828e-11
Epoch 34/100
110/110 [==============================] - ETA: 0s - loss: 1.0885 - accuracy: 0.6037
Epoch 34: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 476ms/step - loss: 1.0885 - accuracy: 0.6037 - val_loss: 1.0639 - val_accuracy: 0.6250 - lr: 4.8828e-12
Epoch 35/100
110/110 [==============================] - ETA: 0s - loss: 1.0676 - accuracy: 0.6108
Epoch 35: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 475ms/step - loss: 1.0676 - accuracy: 0.6108 - val_loss: 1.0642 - val_accuracy: 0.6239 - lr: 4.8828e-13
Epoch 36/100
110/110 [==============================] - ETA: 0s - loss: 1.0833 - accuracy: 0.6071
Epoch 36: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 476ms/step - loss: 1.0833 - accuracy: 0.6071 - val_loss: 1.0642 - val_accuracy: 0.6239 - lr: 4.8828e-14
Epoch 37/100
110/110 [==============================] - ETA: 0s - loss: 1.0671 - accuracy: 0.6091
Epoch 37: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 475ms/step - loss: 1.0671 - accuracy: 0.6091 - val_loss: 1.0641 - val_accuracy: 0.6239 - lr: 4.8828e-15
Epoch 38/100
110/110 [==============================] - ETA: 0s - loss: 1.0618 - accuracy: 0.6202
Epoch 38: val_loss did not improve from 1.06351
110/110 [==============================] - 53s 477ms/step - loss: 1.0618 - accuracy: 0.6202 - val_loss: 1.0640 - val_accuracy: 0.6239 - lr: 4.8828e-16
Epoch 39/100
110/110 [==============================] - ETA: 0s - loss: 1.0760 - accuracy: 0.6193
Epoch 39: val_loss did not improve from 1.06351
110/110 [==============================] - 53s 477ms/step - loss: 1.0760 - accuracy: 0.6193 - val_loss: 1.0640 - val_accuracy: 0.6250 - lr: 4.8828e-17
Epoch 40/100
110/110 [==============================] - ETA: 0s - loss: 1.0742 - accuracy: 0.6085
Epoch 40: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0742 - accuracy: 0.6085 - val_loss: 1.0640 - val_accuracy: 0.6261 - lr: 4.8828e-18
Epoch 41/100
110/110 [==============================] - ETA: 0s - loss: 1.0736 - accuracy: 0.6145
Epoch 41: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0736 - accuracy: 0.6145 - val_loss: 1.0639 - val_accuracy: 0.6239 - lr: 4.8828e-19
Epoch 42/100
110/110 [==============================] - ETA: 0s - loss: 1.0590 - accuracy: 0.6156
Epoch 42: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0590 - accuracy: 0.6156 - val_loss: 1.0639 - val_accuracy: 0.6273 - lr: 4.8828e-20
Epoch 43/100
110/110 [==============================] - ETA: 0s - loss: 1.0722 - accuracy: 0.6114
Epoch 43: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0722 - accuracy: 0.6114 - val_loss: 1.0638 - val_accuracy: 0.6273 - lr: 4.8828e-21
Epoch 44/100
110/110 [==============================] - ETA: 0s - loss: 1.0961 - accuracy: 0.6017
Epoch 44: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0961 - accuracy: 0.6017 - val_loss: 1.0642 - val_accuracy: 0.6250 - lr: 4.8828e-22
Epoch 45/100
110/110 [==============================] - ETA: 0s - loss: 1.0809 - accuracy: 0.6125
Epoch 45: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 475ms/step - loss: 1.0809 - accuracy: 0.6125 - val_loss: 1.0641 - val_accuracy: 0.6239 - lr: 4.8828e-23
Epoch 46/100
110/110 [==============================] - ETA: 0s - loss: 1.0769 - accuracy: 0.6170
Epoch 46: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 475ms/step - loss: 1.0769 - accuracy: 0.6170 - val_loss: 1.0645 - val_accuracy: 0.6239 - lr: 4.8828e-24
Epoch 47/100
110/110 [==============================] - ETA: 0s - loss: 1.0732 - accuracy: 0.6125
Epoch 47: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0732 - accuracy: 0.6125 - val_loss: 1.0640 - val_accuracy: 0.6261 - lr: 4.8828e-25
Epoch 48/100
110/110 [==============================] - ETA: 0s - loss: 1.0904 - accuracy: 0.6176
Epoch 48: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0904 - accuracy: 0.6176 - val_loss: 1.0642 - val_accuracy: 0.6250 - lr: 4.8828e-26
Epoch 49/100
110/110 [==============================] - ETA: 0s - loss: 1.0692 - accuracy: 0.6151Restoring model weights from the end of the best epoch: 29.

Epoch 49: val_loss did not improve from 1.06351
110/110 [==============================] - 52s 474ms/step - loss: 1.0692 - accuracy: 0.6151 - val_loss: 1.0646 - val_accuracy: 0.6239 - lr: 4.8828e-27
Epoch 49: early stopping
35/35 [==============================] - 9s 241ms/step

Классификационный отчет:
              precision    recall  f1-score   support

       Beige       0.61      0.71      0.66       100
       Black       0.49      0.57      0.53       100
        Blue       0.58      0.74      0.65       100
       Brown       0.53      0.49      0.51       100
       Green       0.53      0.28      0.37       100
        Grey       0.39      0.28      0.33       100
      Orange       0.58      0.61      0.60       100
         Red       0.71      0.65      0.68       100
      Silver       0.59      0.62      0.60       100
       White       0.79      0.85      0.82       100
      Yellow       0.76      0.87      0.81       100

    accuracy                           0.70       910
   macro avg       0.71      0.71      0.70       910
weighted avg       0.72      0.70      0.71       910


F1 Macro: 0.7038
"""

## Итоги по второму эксперименту
### F1 Macro: 0.7038
![Графики обучения модели](plots/graph_pretrained_blocked.jpg)
![Матрица ошибок](plots/matrix_custom.jpg)


# Выводы
## Сравнение результатов
- Результат первого эксперимента = 0.7454
- Результат второго эксперимента = 0.7038

Эксперимент показал, предобученная модель показала себя хуже, чем модель с нуля.

Это может быть связано с тем, что предобученная модель обучалась на других данных и не смогла нормально адаптироваться к новым данным.

## Подход к задаче
Чтобы добиться максимально возможного качества модели прошлось отнормировать данные (привести все возможные цвета к одному значению). Если ограничить данные просто количеством классов с макимальными цветами (например топ 6 или меньше) точность увеличивается и переваливает за 80% (но это не совсем то, что требуется в задаче).

## Оценка качества моделей
Качество моделей на среднем уровне

Это может быть связано с тем, что данные в выборке распределены неравномерно:
- Некоторые цвета (белый, серый и серебристый) можно спутать из-за условий на картинке
- Аугментация данных не сильно помогла улучшить качество модели
- качество картинок может быть низким

Кроме того это может быть связано с тем что отсутствует разметка данных на промежуточных этапах